In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import re

In [2]:
url = 'https://www.moneypartners.co.jp/market/schedule/'
res = requests.get(url)
html_content = res.text
soup = BeautifulSoup(res.content, 'html.parser')

In [3]:
df = pd.DataFrame(columns=['Day', 'Time', 'Importance', 'Country', 'Title', 'Last', 'Predict', 'Result'])


day_tags = soup.find_all('td', {'align': 'center', 'class': 'row'})
importance_tags = soup.find_all('td', {'align': 'left'})
name_tags = soup.find_all(['td'], align=None)
num_tags = soup.find_all('td', {'align': 'center', 'rowspan': None, 'class': None}) #3つ（時刻、前回、予想）
result_tags = soup.find_all('td', {'align': 'center', 'rowspan': None, 'class': 'result'})

for i in range(len(name_tags)):
    df.loc[i, 'Time'] = re.sub(r'\s', '', num_tags[3*i+0].text)
    df.loc[i, 'Title'] = re.sub(r'\s', '', name_tags[i].text)
    df.loc[i, 'Importance'] = importance_tags[i].img['src']
    df.loc[i, 'Last'] = re.sub(r'\s', '', num_tags[3*i+1].text)
    df.loc[i, 'Predict'] = re.sub(r'\s', '', num_tags[3*i+2].text)
    df.loc[i, 'Result'] = re.sub(r'\s', '', result_tags[i].text)


day_elements = soup.find_all('td', {'align': 'center', 'class': 'row'})
present_val = 0
former_val = 0

for day_element in day_elements:
    present_val += int(day_element.get('rowspan'))
    df.loc[former_val:present_val, 'Day'] = re.sub(r'\s', '', day_element.text)
    former_val = present_val

In [4]:
df['Importance'] = df['Importance'].str.replace('../../data/market/images/ico_', '')
df['Importance'] = df['Importance'].str.replace('.png', '')
df['Country'] = df['Title'].str.split(")").str[0]
df['Country'] = df['Country'].str.slice(1)
df['Datetime'] = '2023/' +  df['Day'].str.slice(0, -3) + ' ' + df['Time']

# 26時などのデータがあるので、df['Datetime']をdatetimeにする場合はデータの修正が必要と思われる
# s_format = '%Y/%m/%d %H:%M'
# df['Datetime'] = datetime.strptime(df['Datetime'], s_format)
# df['Datetime'] = pd.to_datetime(df['Datetime'], format=s_format)

In [5]:
df

,Day,Time,Importance,Country,Title,Last,Predict,Result,Datetime
0,7/31(月),08:50,etc,日,(日)6月小売業販売額[前年同月比],5.7％(5.8％),5.4％,5.9％,2023/7/31 08:50
1,7/31(月),08:50,etc,日,(日)6月百貨店・スーパー販売額（既存店）[前年同月比],3.4％,―,4.1％,2023/7/31 08:50
2,7/31(月),08:50,low,日,(日)6月鉱工業生産・速報値[前月比],-2.2％,2.4％,2.0％,2023/7/31 08:50
3,7/31(月),08:50,low,日,(日)6月鉱工業生産・速報値[前年同月比],4.2％,0.3％,-0.4％,2023/7/31 08:50
4,7/31(月),10:00,etc,NZ,(NZ)7月ＡＮＺ企業信頼感,-18.0,―,-13.1,2023/7/31 10:00
...,...,...,...,...,...,...,...,...,...
386,9/1(金),21:30,normal,米,(米)8月平均時給[前年同月比],4.4％,―,,2023/9/1 21:30
387,9/1(金),22:45,low,米,(米)8月製造業購買担当者景気指数（ＰＭＩ、改定値）,―,―,,2023/9/1 22:45
388,9/1(金),23:00,normal,米,(米)8月ＩＳＭ製造業景況指数,46.4,―,,2023/9/1 23:00
389,9/1(金),23:00,etc,米,(米)7月建設支出[前月比],0.5％,―,,2023/9/1 23:00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 0 to 390
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Day         391 non-null    object
 1   Time        391 non-null    object
 2   Importance  391 non-null    object
 3   Country     391 non-null    object
 4   Title       391 non-null    object
 5   Last        391 non-null    object
 6   Predict     391 non-null    object
 7   Result      391 non-null    object
 8   Datetime    391 non-null    object
dtypes: object(9)
memory usage: 38.6+ KB
